In [206]:
import pandas as pd 
import re
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.style.use('seaborn-whitegrid')
from PIL import Image

from load_classes import fetch_classdict

In [2]:
class_dict = fetch_classdict()
class_dict

{'N': 'Normal',
 'D': 'Diabetes',
 'G': 'Glaucoma',
 'C': 'Cataract',
 'A': 'Age',
 'H': 'Hypertension',
 'M': 'Pathological',
 'O': 'Other'}

###  Picking Useful Classes

In [15]:
# classes of significance to this project: Classes of Interest (CoI)
'''
Given the amount of time that we have, constructing a proprietary network able to deal with
all of the included will take a lot of RnD.
'''
class_dict
# coi = ['Normal', 'Diabetes', 'Glaucoma', 'Cataract','Pathological']
coi = ['N','D','G','C','M']
coi_dict = {}
for i in coi:
    coi_dict[i] = class_dict[i]
coi_dict

{'N': 'Normal',
 'D': 'Diabetes',
 'G': 'Glaucoma',
 'C': 'Cataract',
 'M': 'Pathological'}

In [22]:
# record coi
with open('coi.txt','w') as f:
    template = ''
    for key, value in zip(coi_dict.keys(),coi_dict.values()):
        template += f'{value} ({key}),\n' 
    f.write(template)
    

### ToDo
- [x] Engineer the diagnosis/the fundus filenames which correspond to a certain disease.
- [ ] Create a two more columns which describe the diagnosis, columns should be left-diagnosis and right-diagnosis.
- [ ] Engineer the paths from the lists obtained.
- [ ] Merge both Left and Right Fundii of dieseases into a single folder titled after that diagnosis 
- [ ] Using the paths, make distributions of Both Normal and the [X] Diagnosis into repositories structured in convinience to the acceptable tensorflow format
- [ ] Distribute and balance the data according to the difference between Normal class and the actual class

### Engineering Left and Right Fundus Diagnosis from Keywords

In [101]:
print(coi_dict)
class_dict

{'N': 'Normal', 'D': 'Diabetes', 'G': 'Glaucoma', 'C': 'Cataract', 'M': 'Pathological'}


{'N': 'Normal',
 'D': 'Diabetes',
 'G': 'Glaucoma',
 'C': 'Cataract',
 'A': 'Age',
 'H': 'Hypertension',
 'M': 'Pathological',
 'O': 'Other'}

In [180]:
xdf = pd.read_excel('../../ODIR-data/ODIR-5K/ODIR-5K/data.xlsx')
metadf = pd.read_csv('../../ODIR-data/full_df.csv')

In [182]:
# df = xdf.copy(deep=True)
df = metadf.copy(deep=True)
df.head(2)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg


In [183]:
# Studying Patterns for the classes of interest 
# df[df['Right-Diagnostic Keywords'].str.contains('pathological')][['Right-Diagnostic Keywords','labels']]

In [184]:
# test
if re.search(r'glaucoma','dry age-related macular degeneration，glaucoma'):
    print(True)

True


In [185]:
# Masking Functions
def has_pathological(text):
    if re.search(r'pathological',text):
        return 1
    else:
        return 0

def has_glaucoma(text):
    if re.search(r'glaucoma',text):
        return 1
    else:
        return 0
    
def has_diabetis(text):
    if re.search(r'[non]{3}?\s?proliferative retinopathy',text):
        return 1
    else:
        return 0
    
def has_cataract(text):
    if "cataract" in text:
        return 1
    else:
        return 0

def has_normal(text):
    if "normal" in text:
        return 1
    else:
        return 0
    
df["left_pathological"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_pathological(x))
df["right_pathological"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_pathological(x))

df["left_glaucoma"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_glaucoma(x))
df["right_glaucoma"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_glaucoma(x))

df["left_diabetis"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_diabetis(x))
df["right_diabetis"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_diabetis(x))

df["left_cataract"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_cataract(x))
df["right_cataract"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_cataract(x))

df["left_normal"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_normal(x))
df["right_normal"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_normal(x))

In [188]:
df.head(2)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,...,left_pathological,right_pathological,left_glaucoma,right_glaucoma,left_diabetis,right_diabetis,left_cataract,right_cataract,left_normal,right_normal
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,...,0,0,0,0,0,0,0,0,1,1


In [190]:
# Collecting Images which are correlated to their corresponding diagnosis

left_cataract = df.loc[(df.C ==1) & (df.left_cataract == 1)]["Left-Fundus"].values
right_cataract = df.loc[(df.C ==1) & (df.right_cataract == 1)]["Right-Fundus"].values

left_normal = df.loc[(df.N ==1) & (df.left_normal == 1)]["Left-Fundus"].values
right_normal = df.loc[(df.N ==1) & (df.right_normal == 1)]["Right-Fundus"].values

left_diabetic = df.loc[(df.D ==1) & (df.left_diabetis == 1)]["Left-Fundus"].values
right_diabetic = df.loc[(df.D ==1) & (df.right_diabetis == 1)]["Right-Fundus"].values

left_glaucoma = df.loc[(df.G ==1) & (df.left_glaucoma == 1)]["Left-Fundus"].values
right_glaucoma = df.loc[(df.G ==1) & (df.right_glaucoma == 1)]["Right-Fundus"].values

left_myopia = df.loc[(df.M ==1) & (df.left_pathological == 1)]["Left-Fundus"].values
right_myopia = df.loc[(df.M ==1) & (df.right_pathological == 1)]["Right-Fundus"].values


In [191]:
order = ['Cataract', 'Diabetic Retinopathy', 'Glaucoma', 'Pathological Myopia', 'Normal']
left_samples = [left_cataract,left_diabetic,left_glaucoma, left_myopia, left_normal]
right_samples = [right_cataract, right_diabetic, right_glaucoma, right_myopia, right_normal]
left_dict = {}
right_dict = {}
# making dicts
for clsname, var in zip(order, left_samples):
    left_dict[clsname] = var
for clsname, var in zip(order, right_samples):
    right_dict[clsname] = var


In [205]:
print('LEFT FUNDUS SAMPLES')
for key, value in zip(left_dict.keys(), left_dict.values()):
    print(f'{key}, samples: {len(value)}')
print('*'*50)

print('RIGHT FUNDUS SAMPLES')
for key, value in zip(right_dict.keys(), right_dict.values()):
    print(f'{key}, samples: {len(value)}')
print('*'*50)

# Total Samples
print('TOTAL SAMPLES - LEFT+RIGHT')
total_sum = 0
for key, lvalue, _, rvalue in zip(left_dict.keys(), left_dict.values(),right_dict.keys(), right_dict.values()):
    total_sum += len(lvalue)+len(rvalue)
    print(f'{key}, samples: {len(lvalue)+len(rvalue)}')
print('*'*50)
print(f'TOTAL SAMPLES OF ALL CLASSES - {total_sum}')

LEFT FUNDUS SAMPLES
Cataract, samples: 304
Diabetic Retinopathy, samples: 1588
Glaucoma, samples: 332
Pathological Myopia, samples: 216
Normal, samples: 2100
**************************************************
RIGHT FUNDUS SAMPLES
Cataract, samples: 290
Diabetic Retinopathy, samples: 1658
Glaucoma, samples: 284
Pathological Myopia, samples: 241
Normal, samples: 2101
**************************************************
TOTAL SAMPLES - LEFT+RIGHT
Cataract, samples: 594
Diabetic Retinopathy, samples: 3246
Glaucoma, samples: 616
Pathological Myopia, samples: 457
Normal, samples: 4201
**************************************************
TOTAL SAMPLES OF ALL CLASSES - 9114


In [208]:
# saving progress
df.to_csv('engineered.csv')

In [209]:
df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,...,left_pathological,right_pathological,left_glaucoma,right_glaucoma,left_diabetis,right_diabetis,left_cataract,right_cataract,left_normal,right_normal
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,...,0,0,0,0,0,0,0,0,1,1
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,0,1,1,0,0,0,0
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6387,4686,63,Male,4686_left.jpg,4686_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,...,0,0,0,0,1,0,0,0,0,0
6388,4688,42,Male,4688_left.jpg,4688_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,...,0,0,0,0,1,1,0,0,0,0
6389,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,...,0,0,0,0,1,0,0,0,0,1
6390,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,...,0,0,0,0,1,1,0,0,0,0
